# Get Clinical Notes of Cohort
This Notebook is extracting the Clinical Notes for an specific set of MRN. This Clinical Notes will be used to complete the Phenotyping of the patient. 

In [ ]:
import re
import os
import re
import json
import spacy
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm_notebook
from hdbcli import dbapi
import mysql.connector

In [ ]:
#Connection to HANA
conn_hana = dbapi.connect(
    address='10.111.170.19',
    port=39041,
    user='', # put your username
    password='' # put your password
)

cursor = conn_hana.cursor()

In [ ]:
def read_json_as_dict(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data
# fetches all clinical notes from given patients MRNs.
# it joins other tables to get more additional data of the clinical notes
def get_patients_notes_hana(patatiens_mrns, con):
    mrns_str = ', '.join(patatiens_mrns.map(str)) 
    notes_df = pd.read_sql('SELECT na.NOTE_ID, na.TERM_SEQUENCE, nt.MEDICAL_RECORD_NUMBER, nt.AGE_IN_DAYS, nt.NOTE_TYPE ' 
            'FROM MSDW_2018.NOTE_ANNOTATION AS na ' 
            'INNER JOIN MSDW_2018.NOTE_TYPE AS nt ON na.NOTE_ID = nt.NOTE_ID '
            'WHERE nt.MEDICAL_RECORD_NUMBER IN ({})'.format(mrns_str), con=con)
    #return decipher_notes(notes_df)
    return notes_df

# fetches all clinical notes from given patients MRNs.
# it joins other tables to get more additional data of the clinical notes
def get_patients_notes_sql(patatiens_mrns, con):
    mrns_str = ', '.join(patatiens_mrns.apply(str)) 
    notes_df = pd.read_sql('SELECT na.NOTE_ID, na.TERM_SEQUENCE, nt.MEDICAL_RECORD_NUMBER, nt.AGE_IN_DAYS, nt.NOTE_TYPE ' 
            'FROM note_annotation AS na ' 
            'INNER JOIN note_type AS nt ON na.NOTE_ID = nt.NOTE_ID '
            'WHERE nt.MEDICAL_RECORD_NUMBER IN ({})'.format(mrns_str), con=con)
    #return decipher_notes(notes_df)
    return notes_df

In [ ]:
#Load Cohort
HF_Cohort = pq.read_table('Unique_HF_ICD_Cohort.parquet').to_pandas()

#HF_Cohort=HF_Cohort.head(100)
#sort MRN descendind 
HF_Cohort=HF_Cohort.sort_values(by='medical_record_number', ascending=False)
HF_Cohort

In [ ]:
#get 10.000 and delete the subset of the whole frame
HF_Cohort_10k=HF_Cohort.head(10000)
HF_Cohort=HF_Cohort.drop(HF_Cohort_10k.index)

In [ ]:
HF_Cohort

In [ ]:
#Fetch clinical Notes of Cohort: 

mrns = HF_Cohort_10k.index
notes_df = get_patients_notes_hana(patatiens_mrns=mrns, con=conn_hana)
notes_df.columns = [x.lower() for x in notes_df.columns]
notes_df
notes_df.to_parquet('ALL_HF_ICD_Notes_Cohort8.parquet')

In [ ]:
notes_df

In [ ]:
#combnine the different dataframes: 
#Load all 8 Parts: 
Complete_Notes_Cohort = pq.read_table('ALL_HF_ICD_Notes_Cohort1.parquet').to_pandas()
Notes_Cohort2 = pq.read_table('ALL_HF_ICD_Notes_Cohort2.parquet').to_pandas()
Notes_Cohort3 = pq.read_table('ALL_HF_ICD_Notes_Cohort3.parquet').to_pandas()
Notes_Cohort4 = pq.read_table('ALL_HF_ICD_Notes_Cohort4.parquet').to_pandas()
Notes_Cohort5 = pq.read_table('ALL_HF_ICD_Notes_Cohort5.parquet').to_pandas()
Notes_Cohort6 = pq.read_table('ALL_HF_ICD_Notes_Cohort6.parquet').to_pandas()
Notes_Cohort7 = pq.read_table('ALL_HF_ICD_Notes_Cohort7.parquet').to_pandas()
Notes_Cohort8 = pq.read_table('ALL_HF_ICD_Notes_Cohort8.parquet').to_pandas()
Complete_Notes_Cohort=pd.concat([Complete_Notes_Cohort,Notes_Cohort2,Notes_Cohort3,Notes_Cohort4,Notes_Cohort5,Notes_Cohort6,Notes_Cohort7,Notes_Cohort8])
Complete_Notes_Cohort

In [ ]:
Complete_Notes_Cohort.to_parquet('ALL_HF_ICD_Notes_Cohort.parquet')

In [ ]:
#define Vocab ID that we want to look up: 'multi-organ failure','cardiac failure','chronic heart failure','ventricular failure','heart failure'
vocab_ID= set(['33576','42931','48256','54066','1736'])

In [ ]:
#Filter CLinical Notes which include Terms that are related to HF 
notes_MRN_df= pd.DataFrame(columns=['MRN','Note_ID','age_in_days','Term'])
for index,row in Complete_Notes_Cohort.iterrows(): 
    #print(index , row['term_sequence'])
    term_sequence=row['term_sequence']
    #term_sequence= "|"+term_sequence+"|"
    terms=term_sequence.split('|')
    for term in terms: 
        if term in vocab_ID: 
            print("YES",row['medical_record_number'], term)
            new_row = {'MRN':row['medical_record_number'],'Note_ID':row['note_id'],'age_in_days':row['age_in_days'],'Term':term}
            notes_MRN_df= notes_MRN_df.append(new_row, ignore_index=True)

In [ ]:
notes_MRN_df.to_parquet('ALL_Matches_HF_ICD_Notes_Cohort.parquet')


In [ ]:
notes_MRN_df

In [ ]:
notes_MRN_df_unique=notes_MRN_df.loc[~notes_MRN_df["MRN"].duplicated(keep='first')]

In [ ]:
notes_MRN_df_unique

In [ ]:
notes_MRN_df_unique.to_parquet('Unique_Matches_HF_ICD_Notes_Cohort.parquet')

# END of Notebook

In [ ]:
#get ICD Code and terms which are within 100 days
#load cohort: 
HF_Cohort = pq.read_table('Final_HF_ICD_EF_Cohort.parquet').to_pandas()
#load clinical notes
notes_MRN_df = pq.read_table('ALL_Matches_Clinical_Notes.parquet').to_pandas()

In [ ]:
notes_MRN_df


In [ ]:
mrn= '2927854333'
print(notes_MRN_df.loc[notes_MRN_df['MRN'] == int(mrn)])

In [ ]:
notes_ICD_df= pd.DataFrame(columns=['MRN','age_in_days_icd','Note_ID','age_in_days','Term'])
for index,row_icd in HF_Cohort.iterrows(): 
    mrn= index
    age_in_days_icd= row_icd['age_in_days_x']
    if notes_MRN_df.loc[notes_MRN_df['MRN'] == int(mrn)].empty==False:
        #print(notes_MRN_df.loc[notes_MRN_df['MRN'] == int(mrn)])
        notes_specific_MRN= notes_MRN_df.loc[notes_MRN_df['MRN'] == int(mrn)]
        for index,row_note in notes_specific_MRN.iterrows(): 
            age_in_days_note= row_note['age_in_days']
            if abs(age_in_days_note-age_in_days_icd)<=100: 
                print(age_in_days_icd)
                new_row  = {'MRN':row_note['MRN'],'age_in_days_icd': age_in_days_icd,'Note_ID':row_note['Note_ID'],'age_in_days': age_in_days_note, 'Term':row_note['Term']}
                notes_ICD_df= notes_ICD_df.append(new_row, ignore_index=True)
                
        
    

In [ ]:
notes_ICD_df

In [ ]:
notes_ICD_df_unique_100days=notes_ICD_df.loc[~notes_ICD_df["MRN"].duplicated(keep='first')]

In [ ]:
notes_ICD_df_unique_100days